In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

This code is written as part of COMP8755- Individual Computing Project

This code is for preprocessing and training the LSTM network

The songs are first parsed and the notes of different instruments are procesed to train and mdh5 files for music generation.


@Author:
Mithun
u6849970

@ Supervisor:
Professor Nick Birbillis

"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


In [ ]:
from music21 import *
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils

In [ ]:
"""
This function is for reading the midi files which we feed as a data set and 
parse it into 3 categories, chords, rests and notes
"""
def read_midi(file):
    print("Loading Music File:",file)
    
    notes=[]
    notes_to_parse = None
    rest = True
    #parsing a midi file
    midi = converter.parse(file)
    s2 = instrument.partitionByInstrument(midi)
    print("Number of instrument parts: " + str(len(s2.parts)))
    for part in s2.parts:
    
        #select elements of only piano
             
        notes_to_parse = part.recurse() 
      
        #finding whether a particular element is note or a chord    
        for element in notes_to_parse:
                
                #note
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
                #rest
            elif isinstance(element, note.Rest) and rest:
                notes.append("rest")                
                #chord
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return np.array(notes)

    return notes

In [ ]:
path='midi/'

#read all the filenames
files=[i for i in os.listdir(path) if i.endswith(".mid")]

#reading each midi file
notes_array = np.array([read_midi(path+i) for i in files])

In [ ]:
#converting 2D array into a vector
notes_ = [element for note_ in notes_array for element in note_]

#No. of unique notes
unique_notes = list(set(notes_))
print(len(unique_notes))

In [ ]:
#importing library
from collections import Counter

#computing frequency of each note
freq = dict(Counter(notes_))

#library for visualiation
import matplotlib.pyplot as plt

#consider only the frequencies
no=[count for _,count in freq.items()]

#set the figure size
plt.figure(figsize=(5,5))

#plot
plt.hist(no)

In [ ]:
frequent_notes = [note_ for note_, count in freq.items() if count>=60]
print(len(frequent_notes))

In [ ]:
new_music=[]

for notes in notes_array:
    temp=[]
    for note_ in notes:
        if note_ in frequent_notes:
            temp.append(note_)            
    new_music.append(temp)
    
new_music = np.array(new_music)

In [ ]:
no_of_timesteps = 32
x = []
y = []

for note_ in new_music:
    for i in range(0, len(note_) - no_of_timesteps, 1):
        
        #preparing input and output sequences
        input_ = note_[i:i + no_of_timesteps]
        output = note_[i + no_of_timesteps]
        
        x.append(input_)
        y.append(output)
        
x=np.array(x)
y=np.array(y)

In [ ]:
unique_x = list(set(x.ravel()))
x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

In [ ]:
#preparing input sequences
x_seq=[]
for i in x:
    temp=[]
    for j in i:
        #assigning unique integer to every note
        temp.append(x_note_to_int[j])
    x_seq.append(temp)
    
x_seq = np.array(x_seq)

In [ ]:
unique_y = list(set(y))
y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y)) 
y_seq=np.array([y_note_to_int[i] for i in y])

In [ ]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)

In [ ]:
"""
we need to create the network back so that we can predict the next set of notes for generating music
"""

In [ ]:
def lstm():
  model = Sequential()
  model.add(LSTM(128,return_sequences=True))
  model.add(LSTM(512))
  model.add(Dense(256))
  model.add(Activation('relu'))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
  return model

In [ ]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
import keras.backend as K

K.clear_session()
model = Sequential()
    
#embedding layer
model.add(Embedding(len(unique_x), 100, input_length=32,trainable=True)) 

model.add(Conv1D(64,3, padding='causal',activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
    
model.add(Conv1D(128,3,activation='relu',dilation_rate=2,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

model.add(Conv1D(256,3,activation='relu',dilation_rate=4,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
              
model.add(GlobalMaxPool1D())
    
model.add(Dense(256, activation='relu'))
model.add(Dense(len(unique_y), activation='softmax'))
    
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.summary()

In [ ]:
"""
This is the checkpint for training the model, after every epoch,
the model is updated. We can stop the model once the loss value has saturated
"""
from keras.callbacks import ModelCheckpoint
mc= ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss',
    mode='min', 
    save_best_only=True,
    verbose=1)


In [ ]:

history = model.fit(np.array(x_tr),np.array(y_tr),batch_size=128,epochs=200, validation_data=(np.array(x_val),np.array(y_val)),verbose=1, callbacks=[mc])

In [ ]:
"""
This function generates the notes in a sequential pattern which is then converted to MIDI file
we pick a random sequence from the input and use it as a starting point for generation
"""

In [ ]:
import random
ind = np.random.randint(0,len(x_val)-1)
"""Random number for prediction
"""
random_music = x_val[ind]

predictions=[]
"""
a total of 150 notes are being generated for our dataset,
this value can be changed depending on the dataset.

"""
for i in range(150):

    random_music = random_music.reshape(1,no_of_timesteps)

    #from the prediction input, we use the keras libary for the same

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    #The pattern will contain the musical notes which is stored as an array
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

In [ ]:
x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
predicted_notes = [x_int_to_note[i] for i in predictions]

In [ ]:
"""
Generating the Midi file from prediction
"""

In [ ]:
def convert_to_midi(prediction_output):
   
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                
                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        elif pattern == 'rest':
            new_note = note.Rest()
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # pattern is a note
        else:
            
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.25
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music.mid')

In [ ]:
convert_to_midi(predicted_notes)